In [13]:
import labdataranger as ldr

df = ldr.query.dataframe(
    query = f"""
    MATCH (s:Scan)-[]-(f:Folder)
    WHERE f.name CONTAINS '_Rec'
    MATCH (r:Reconstruction)-[]-(s:Scan)-[]-(a:Acquisition)
    RETURN f.name, s.filepath, a.studyDateAndTime, r.timeAndDate
    """, 
    database='instruments',
    column_map={
    'f.name': 'Name',
    's.filepath': 'Filepath',
    'a.studyDateAndTime': 'Acquired',
    'r.timeAndDate': 'Reconstructed'
    }, verbose=True
) 

Results for query: 
    MATCH (s:Scan)-[]-(f:Folder)
    WHERE f.name CONTAINS '_Rec'
    MATCH (r:Reconstruction)-[]-(...
          Name                                           Filepath  \
0       1L_Rec  /mnt/data/archive/LaraM_Full_Dataset/U37/10_21...   
1       1R_Rec  /mnt/data/archive/LaraM_Full_Dataset/U37/10_21...   
2       NP_Rec  /mnt/data/archive/LaraM_Full_Dataset/U37/10_21...   
3       1L_Rec  /mnt/data/archive/LaraM_Full_Dataset/U37/10_21...   
4       1R_Rec  /mnt/data/archive/LaraM_Full_Dataset/U37/10_21...   
...        ...                                                ...   
2760  2660_Rec  /mnt/data/archive/LaraM_Full_Dataset/02.3.22_7...   
2761  2674_Rec  /mnt/data/archive/LaraM_Full_Dataset/02.3.22_7...   
2762  2933_Rec  /mnt/data/archive/LaraM_Full_Dataset/02.3.22_7...   
2763  2932_Rec  /mnt/data/archive/LaraM_Full_Dataset/02.3.22_7...   
2764    NP_Rec  /mnt/data/archive/LaraM_Full_Dataset/U37/10_21...   

                      Acquired           Reconst

In [ ]:
def query_graph(query, config_file='db_config.json', database='neo4j', verbose=False):
    with open(config_file, 'r') as file:
        config = json.load(file)
    
    uri = config.get("uri")
    username = config.get("username")
    password = config.get("password")
    database = config.get("database", database)

    driver = GraphDatabase.driver(uri, auth=(username, password))
    graph = nx.Graph()
    with driver.session(database=database) as session:
        result = session.run(query)
        for record in result:
            if verbose:
                print(f"  {record}")

            for key, value in record.items():
                if isinstance(value, dict):  # Node
                    graph.add_node(value['id'], **value)
                elif isinstance(value, tuple):  # Relationship
                    start_node, relationship, end_node = value
                    graph.add_node(start_node['id'], **start_node)
                    graph.add_node(end_node['id'], **end_node)
                    graph.add_edge(start_node['id'], end_node['id'], **relationship)

    driver.close()
    
    return graph

# Example usage:
graph_result = query_graph(
    f"""
    MATCH (fp:Folder)-[cf:CONTAINS_FOLDER]->(fc:Folder)
    RETURN fp, cf, fc
    """, database='instruments', verbose=True
)

# Printing nodes and edges of the graph
print("Nodes:")
for node in graph_result.nodes(data=True):
    print(node)

print("\nEdges:")
for edge in graph_result.edges(data=True):
    print(edge)

In [1]:
import labdataranger as ldr

import neo4j

_c = ldr.query.get_db_config()
uri = _c.get("uri")
username = _c.get("username")
password = _c.get("password")
database = 'instruments'

with neo4j.GraphDatabase.driver(uri, auth=(username, password)) as driver:
    graph_result = driver.execute_query(
        """
            MATCH (fp:Folder)-[cf:CONTAINS_FOLDER]->(fc:Folder)
            RETURN fp, cf, fc
        """, 
        database_=database,
        result_transformer_=neo4j.Result.graph)

import pyvis
import networkx as nx

def visualize_result(query_graph, nodes_text_properties):
    visual_graph = pyvis.network.Network(directed=True)

    for node in query_graph.nodes:
        node_label = list(node.labels)[0]
        node_text = node[nodes_text_properties[node_label]]
        visual_graph.add_node(node.element_id, node_text, group=node_label)

    for relationship in query_graph.relationships:
        visual_graph.add_edge(
            relationship.start_node.element_id,
            relationship.end_node.element_id,
            title=relationship.type
        )

    visual_graph.show('network.html', notebook=False)
    
    return visual_graph

nodes_text_properties = {
    "Folder": "folder"
}
pyvis_graph = visualize_result(graph_result, nodes_text_properties)

G = nx.nx_agraph.from_agraph(pyvis_graph)

network.html


AttributeError: 'Network' object has no attribute 'graph_attr'

In [10]:
pyvis_graph.

True

In [11]:
pyvis_graph.

<bound method Network.toggle_stabilization of <class 'pyvis.network.Network'> |N|=7001 |E|=7,000>

In [ ]:
! pip install 